In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Genetic Algorithm for Designing a CNN for MPII Dataset
Based on ideas found in:
    
* https://arxiv.org/pdf/1611.01578.pdf?fbclid=IwAR1YvzhJ_l3tYuHKCRl96UCyKsJc956CipC5FGoVzs0DIIPQNpptjHN6nOM
* https://github.com/PacktPublishing/Hands-On-Genetic-Algorithms-with-Python
* https://github.com/ahmedfgad/GeneticAlgorithmPython

Written by Mark Strefford

MIT License

Copyright © 2021 Timelaps AI Limited

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

* Checkpoint file

In [ ]:
checkpoint = None

In [ ]:
import sys
sys.path.append('../lib')


In [ ]:
import tensorflow as tf
# config = tf.compat.v1.ConfigProto(gpu_options =
#                          tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# # device_count = {'GPU': 1}
# )
# config.gpu_options.allow_growth = True
# session = tf.compat.v1.Session(config=config)
# tf.compat.v1.keras.backend.set_session(session)

In [ ]:
import tensorflow as tf

from objproxies import CallbackProxy
from deap import base, creator, tools

import random
import numpy as np
import pandas as pd
import logging

In [ ]:
from ga_net.train import Trainer
from core.config import config, update_config
from utils.utils import create_logger
from ga import elitism

In [ ]:
# import dataset
from dataset import mpii

* Config

In [ ]:
cfg_name = '../config/watch_ar/ga_cnn/regressor_256x256_d256x3_adam_lr1e-3.yaml'
update_config(cfg_name)

# Required for running in notebook?
config.DATASET.PATH_PREFIX = '../'

logger, output_dir, tb_log_dir = create_logger(
        config, cfg_name, 'train')
logger = logging.getLogger(__name__)

### Define a set of basic layers and config

Image sizes defined in NHWC format for tf.js

In [ ]:
input_shape = (256, 256, 3)

# output_layer_config = [
#         {'name': 'coords_output', 'features': 15, 'activation': 'tanh', 'loss': 'EuclideanLoss'},
#         {'name': 'rotation_output', 'features': 3, 'activation': 'linear', 'loss': 'MSE'},
#         {'name': 'flags_output', 'features': 6, 'activation': 'sigmoid', 'loss': '"BinaryCrossentropy"'}
#     ]

output_layer_config = [
        {'name': 'joints_output', 
         'features': 16 * 3, 
         'activation': 'tanh', 
         'loss': 'EuclideanLoss'},
        {'name': 'joints_vis_output', 
         'features': 16, 
         'activation': 'sigmoid', 
         'loss': '"BinaryCrossentropy"'}
    ]

In [ ]:
POPULATION_SIZE = 20  
MAX_GENERATIONS = 500
P_CROSSOVER = 0.5  # probability for crossover
P_MUTATION = 0.5   # probability for mutating an individual
HALL_OF_FAME_SIZE = 5
CROWDING_FACTOR = 10.0  # crowding factor for crossover and mutation

# From https://stackoverflow.com/questions/58990269/deap-make-mutation-probability-depend-on-generation-number
N_GEN = 1          # Generation counter for mutation
N_EVALS = 0        # Used for changing N_GEN? 

In [ ]:
NUM_LAYERS = 32         # Excluding output layers
NUM_OUTPUT_LAYERS = len(output_layer_config)    # Coords, rotation, flags

LAYER_TYPES = ['DepthwiseConv2D', 'Conv2D', 'Conv2DTranspose']   # 'Dense' etc?
LAYER_TYPE_LOWER = 0.
LAYER_TYPE_UPPER = float(len(LAYER_TYPES))

In [ ]:
FILTER_LOWER_BOUND = 4.
FILTER_UPPER_BOUND = 7.   # 2^4 = 16, but note padding & concat will grow this!
KERNEL_LOWER_BOUND = 1.
KERNEL_UPPER_BOUND = 3.
STRIDE_LOWER_BOUND = 1.
STRIDE_UPPER_BOUND = 2.
INBOUND_CONN_LOWER_BOUND = float(-NUM_LAYERS / 2)   # Force some nodes not to have all links!
INBOUND_CONN_UPPER_BOUND = float(NUM_LAYERS + NUM_OUTPUT_LAYERS)
NUM_SKIP_CONNECTIONS = 2 

In [ ]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7*1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

* Set up dataset for training

In [ ]:
is_gpu = tf.config.list_physical_devices('GPU') != []
dataset = mpii.MPIIDataset(
    config,
    is_train=True,
    random_order=True,
    random_seed=RANDOM_SEED,
    batch_size=1   # 4 if is_gpu else 1  
)
print(f'GPU found = {is_gpu}')

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
tf.config.experimental.list_logical_devices('GPU')

### Setup GA

In [ ]:
BOUNDS_LOW = []
BOUNDS_HIGH = []

for i in range(int(NUM_LAYERS)):
    BOUNDS_LOW.append(LAYER_TYPE_LOWER)
    BOUNDS_LOW.append(FILTER_LOWER_BOUND)
    BOUNDS_LOW.append(KERNEL_LOWER_BOUND)
    BOUNDS_LOW.append(STRIDE_LOWER_BOUND)
    BOUNDS_HIGH.append(len(LAYER_TYPES) - 1)
    BOUNDS_HIGH.append(FILTER_UPPER_BOUND)
    BOUNDS_HIGH.append(KERNEL_UPPER_BOUND)
    BOUNDS_HIGH.append(STRIDE_UPPER_BOUND)
    for j in range (NUM_SKIP_CONNECTIONS):
        BOUNDS_LOW.append(INBOUND_CONN_LOWER_BOUND)   
        BOUNDS_HIGH.append(INBOUND_CONN_UPPER_BOUND)
        
NUM_OF_PARAMS = len(BOUNDS_HIGH)
NUM_CHROMOSOMES = int(NUM_OF_PARAMS / NUM_LAYERS)

In [ ]:
toolbox = base.Toolbox()
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
for i in range(NUM_LAYERS):
    toolbox.register(f'layer_{i}_type_attribute',
                     random.uniform,
                     LAYER_TYPE_LOWER,
                     LAYER_TYPE_UPPER)
    toolbox.register(f'layer_{i}_filters_attribute',
                     random.uniform,
                     FILTER_LOWER_BOUND,
                     FILTER_UPPER_BOUND)
    toolbox.register(f'layer_{i}_kernel_attribute',
                     random.uniform,
                     KERNEL_LOWER_BOUND,
                     KERNEL_UPPER_BOUND)
    toolbox.register(f'layer_{i}_stride_attribute',
                     random.uniform,
                     STRIDE_LOWER_BOUND,
                     STRIDE_UPPER_BOUND)
    for j in range(NUM_SKIP_CONNECTIONS):
        toolbox.register(f'layer_{i}_skip_{j}_attribute',
                         random.uniform,
                         INBOUND_CONN_LOWER_BOUND,
                         float(i))

In [ ]:
layer_attributes = ()

# create a tuple containing n layer_attribute generator for each layer:
layer_size_attributes = ()
for i in range(NUM_LAYERS):
    layer_attributes = layer_attributes + \
                            (toolbox.__getattribute__(f'layer_{i}_type_attribute'),)
    layer_attributes = layer_attributes + \
                            (toolbox.__getattribute__(f'layer_{i}_filters_attribute'),)
    layer_attributes = layer_attributes + \
                            (toolbox.__getattribute__(f'layer_{i}_kernel_attribute'),)
    layer_attributes = layer_attributes + \
                            (toolbox.__getattribute__(f'layer_{i}_stride_attribute'),)
    for j in range(NUM_SKIP_CONNECTIONS):
        layer_attributes = layer_attributes + \
                            (toolbox.__getattribute__(f'layer_{i}_skip_{j}_attribute'),)

* Create the individual operator to full up an individual instance

In [ ]:
toolbox.register("individualCreator",
                 tools.initCycle,
                 creator.Individual,
                 layer_attributes,
                 n=1)

toolbox.register("populationCreator",
                 tools.initRepeat,
                 list,
                 toolbox.individualCreator)

* Create model

In [ ]:
trainer = Trainer(
    RANDOM_SEED, 
    dataset, 
    output_dir, 
    num_chromosomes=NUM_CHROMOSOMES, 
    num_layers=NUM_LAYERS, 
    layer_types=LAYER_TYPES, 
    input_shape=input_shape,
    output_layer_config=output_layer_config,
    merge_type='Concatenate',    # 'Concatenate', 'add',
    debug_net_build=True
)

* Fitness calculation

In [ ]:
def classificationAccuracy(individual):
    return trainer.get_accuracy(individual)

toolbox.register("evaluate", classificationAccuracy)

* Genetic operators

TODO: Are these the most appropriate? 
TODO: Are low and up set correctly? These are the min and max values for all parameters?

In [ ]:
toolbox.register("select", tools.selTournament, tournsize=2)

toolbox.register("mate",
                 tools.cxSimulatedBinaryBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR)

toolbox.register("mutate",
                 tools.mutPolynomialBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR,
                 indpb=CallbackProxy(lambda: 2.0 / N_GEN )
                )

* Create initial population

In [ ]:
if checkpoint:
    # A file name has been given, then load the data from the file
    with open(checkpoint, "r") as cp_file:
        cp = pickle.load(cp_file)
    population = cp["population"]
    start_gen = cp["generation"]
    halloffame = cp["halloffame"]
    logbook = cp["logbook"]
    random.setstate(cp["rndstate"])
else:
    # Start a new evolution
    population = toolbox.populationCreator(n=POPULATION_SIZE)
    start_gen = 0
    hof = tools.HallOfFame(maxsize=HALL_OF_FAME_SIZE)
    logbook = tools.Logbook()

* Prepare the statistics object

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)

* Perform the Genetic Algorithm flow with hof feature added

In [ ]:
population, logbook = elitism.eaSimpleWithElitism(population,
                                                  toolbox,
                                                  cxpb=P_CROSSOVER,
                                                  mutpb=P_MUTATION,
                                                  ngen=MAX_GENERATIONS,
                                                  stats=stats,
                                                  halloffame=hof,
                                                  log_dir=output_dir,
                                                  verbose=True)

* Print best solution found

In [ ]:
print(f'Best solution is: {trainer.format_params(hof.items[0])}')
print(f'Accuracy = {hof.items[0].fitness.values[0]}')